# 01. Tabular Q Learning example

Tabular Q Learning을 실습해봅니다.
- 모든 state의 value function을 table에 저장하고 테이블의 각 요소를 Q Learning으로 업데이트 하는 것으로 학습합니다.

## Colab 용 package 설치 코드

In [ ]:
!pip install gym

### package import

In [1]:
import tensorflow as tf
import numpy as np
import random
import gym
# from gym.wrappers import Monitor

np.random.seed(777)
tf.set_random_seed(777)

print("tensorflow version: ", tf.__version__)
print("gym version: ", gym.__version__)

tensorflow version:  1.8.0
gym version:  0.11.0


## Frozen Lake

**[state]**

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    
**[action]**

    LEFT = 0
    DOWN = 1
    RIGHT = 2
    UP = 3

In [2]:
from IPython.display import clear_output

# Load Environment
env = gym.make("FrozenLake-v0")
# init envrionmnet
env.reset()
# only 'Right' action agent
for _ in range(5):
    env.render()
    env.step(2)


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


### Frozen Lake (not Slippery)

In [3]:
def register_frozen_lake_not_slippery(name):
    from gym.envs.registration import register
    register(
        id=name,
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )

register_frozen_lake_not_slippery('FrozenLakeNotSlippery-v0')

In [4]:
env = gym.make("FrozenLakeNotSlippery-v0")
env.reset()
env.render()
env.step(2)
env.step(2)
env.step(1)
env.step(1)
env.step(1)
_,r,_,_=env.step(2)
env.render()
print(r)


SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
1.0


## Q-Learning
**Pseudo code**  
<img src="./img/qlearning_pseudo.png" width="60%" align="left">  

### Epsilon greedy

In [5]:
# epsilon greedy policy

def epsilon_greedy_action(epsilon, n_action, state, q_table):
        # if epsilon이 random 값보다 클때
        if epsilon > np.random.random():
            # random action
            action = env.action_space.sample()
    
        else:
            # 가장 큰 Q값을 갖는 action을 고른다.
            action = np.argmax(q_table[state, :])
        
        return action

In [6]:
# epsilon greedy test

epsilon = 0
q_table = np.array([[1,0,0,0],
                    [0,0,0,1],
                    [0,1,0,0]])
for state in range(3):
    action = epsilon_greedy_action(epsilon, 4, state, q_table)
    print("state: {}    action: {}".format(state, action))

state: 0    action: 0
state: 1    action: 3
state: 2    action: 1


### Q-value update

In [7]:
def q_update(q_table, state, next_state, action, reward, alpha, gamma):
    
    # 구현해보세요.
    # update 수식은 pseudo code 참조
    q_table[state, action] = q_table[state, action] + \
                            alpha * ((reward + gamma * np.max(q_table[next_state, :])) - q_table[state, action])
    return q_table

In [8]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})

q_table = np.array([[0,0,0,0],
                    [0,1,0,0]], dtype=np.float)
print("start\n", q_table)

reward = 1.0
alpha = 0.1
gamma = 0.9

for i in range(10):
    print("update {}".format(i))
    q_table = q_update(q_table, 0, 1, 2, reward, alpha, gamma)
    print(q_table)

start
 [[ 0.000  0.000  0.000  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 0
[[ 0.000  0.000  0.190  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 1
[[ 0.000  0.000  0.361  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 2
[[ 0.000  0.000  0.515  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 3
[[ 0.000  0.000  0.653  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 4
[[ 0.000  0.000  0.778  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 5
[[ 0.000  0.000  0.890  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 6
[[ 0.000  0.000  0.991  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 7
[[ 0.000  0.000  1.082  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 8
[[ 0.000  0.000  1.164  0.000]
 [ 0.000  1.000  0.000  0.000]]
update 9
[[ 0.000  0.000  1.238  0.000]
 [ 0.000  1.000  0.000  0.000]]


### Agent class

In [9]:
class Tabular_Q_agent:
    def __init__(self, q_table, n_action, epsilon, alpha, gamma):
        self.q_table = q_table
        self.epsilon = epsilon
        self.alpha = alpha
        self.gamma = gamma
        self.n_action = n_action
    
    def get_action(self, state):
    
        if self.epsilon > np.random.random():
            action = env.action_space.sample()
    
        else:
            # 가장 큰 Q값을 갖는 action을 고른다. 같은 action이 있으면 랜덤으로.
            action = np.argmax(self.q_table[state, :])
            # action = np.random.choice(np.flatnonzero(self.q_table[state, :] == self.q_table[state, :].max()))
            
        if self.epsilon > 0.01:
            self.epsilon -= 0.001
        
        return action
    
    def q_update(self, state, next_state, action, reward):
    
        self.q_table[state, action] = self.q_table[state, action] + \
                                               self.alpha * ((reward + self.gamma * np.max(self.q_table[next_state, :])) - self.q_table[state, action])
        

## Goal에 도착하기 위해 생각해야 하는것
1. Goal에 한번이라도 도착해야만 reward가 나와서 update 된다 $\rightarrow$ goal에 어떻게 가게 할까?
2. np.argmax로 Q값이 가장 큰 것을 고르는데 같은 Q값일 경우 무조껀 작은 index를 고른다. $\rightarrow$ 같은 Q값일 경우 랜덤하게 고르게 해야 exploration 한다.
3. hole에 빠졌을 때 episode가 끝나긴 하지만 reward에 차이는 없다. $\rightarrow$ hole에 빠져서 끝나면 negative reward를 주도록 한다.
4. 학습이 잘 되어도 epsilon 만큼의 확률로 random action을 한다. $\rightarrow$ 학습이 진행될수록 epsilon을 줄인다.

### Training agent

In [11]:
env = gym.make("FrozenLakeNotSlippery-v0")

EPISODE = 500
epsilon = 0.9
alpha = 0.8 # learning rate
gamma = 0.9 # discount factor
n_action = env.action_space.n

rlist = []
slist = []

is_render = False

# initialize Q-Table 
q_table = np.random.rand(env.observation_space.n, env.action_space.n)
print("Q table size: ", q_table.shape)

# agent 생성
agent = Tabular_Q_agent(q_table, n_action, epsilon, alpha, gamma)

for e in range(EPISODE):
    state = env.reset()
    print("[Episode {}]".format(e))
    if is_render:
        env.render()
    
    total_reward = 0
    goal = 0
    done = False
    limit = 0
    while not done and limit < 99:
        # 1. select action by e-greedy policy
        action = agent.get_action(state)
            
        # 2. do action and go to next state
        # env.step()을 사용해 1 step 이동 후 next state와 reward, done 값을 받아옴.
        next_state, reward, done, _ = env.step(action)
        if is_render:
            env.render()
            
        # 2.1. hole 에 빠졌을 때 (-) reward를 받도록 함.      
        if reward == 1.0:
            print("GOAL")
            goal = 1
        elif done:
            reward = reward - 1
        
        # 3. Q update
        agent.q_update(state, next_state, action, reward)
        
        slist.append(state)
        state = next_state
        
        total_reward += reward
        limit += 1
        
    print(slist)
    slist = []
    print("total reward: ", total_reward)
    rlist.append(goal)
    
print("성공한 확률" + str(sum(rlist) / EPISODE) + "%")

Q table size:  (16, 4)
[Episode 0]
[0, 0, 0, 1, 0, 1]
total reward:  -1.0
[Episode 1]
GOAL
[0, 4, 8, 4, 8, 8, 9, 10, 14, 14, 14, 14, 14]
total reward:  1.0
[Episode 2]
[0, 0, 1, 0, 4, 8, 4]
total reward:  -1.0
[Episode 3]
[0, 4, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 4]
total reward:  -1.0
[Episode 4]
[0, 1, 2, 2, 2, 3, 3, 3, 2, 2, 2, 3, 3, 2, 2, 3]
total reward:  -1.0
[Episode 5]
[0, 4, 8, 4, 0, 0, 1, 2, 2, 1]
total reward:  -1.0
[Episode 6]
[0, 0, 0, 1, 0, 4, 8, 9, 8, 9, 10]
total reward:  -1.0
[Episode 7]
[0, 0, 0, 4, 4, 0, 0, 4]
total reward:  -1.0
[Episode 8]
[0, 0, 1, 1]
total reward:  -1.0
[Episode 9]
[0, 0, 1]
total reward:  -1.0
[Episode 10]
[0, 0, 0, 0, 0, 4]
total reward:  -1.0
[Episode 11]
[0, 4]
total reward:  -1.0
[Episode 12]
[0, 4, 0, 0, 0, 0, 0, 4, 0, 0, 4, 8, 9, 10, 14, 10]
total reward:  -1.0
[Episode 13]
[0, 4, 4]
total reward:  -1.0
[Episode 14]
[0, 4, 0, 4, 4, 8, 9, 13]
total reward:  -1.0
[Episode 15]
[0, 0, 0, 0, 4, 8]
total reward:  -1.0
[Episode 16]
[0, 4, 8]


GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 302]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 303]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 304]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 305]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 306]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 307]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 308]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 309]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 310]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 311]
GOAL
[0, 4, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 312]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 313]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 314]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 315]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 316]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode 317]
GOAL
[0, 4, 8, 9, 10, 14]
total reward:  1.0
[Episode

In [11]:
np.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print(agent.q_table)

[[ 0.220  0.570  1.035  0.903]
 [ 0.916 -0.205  1.150  1.003]
 [ 0.266  1.277  0.790  1.140]
 [ 1.112 -0.458  0.255  0.255]
 [ 0.242  0.990 -0.205  0.225]
 [ 0.193  0.611  0.883  0.622]
 [-0.205  1.419 -0.460  1.143]
 [ 0.517  0.518  0.600  0.533]
 [ 0.267 -0.387  1.255  0.261]
 [ 0.291  1.419  1.194 -0.205]
 [ 1.251  1.577 -0.287  1.006]
 [ 0.551  0.472  0.792  0.115]
 [ 0.681  0.362  0.344  0.450]
 [-0.386  0.296  1.577  0.300]
 [ 1.197  0.323  1.752  0.328]
 [ 0.016  0.097  0.693  0.836]]


### Test agent

In [12]:
state = env.reset()
done = False
limit = 0

agent.epsilon = 0.0
while not done and limit < 30:
    action = agent.get_action(state)
    next_state, reward, done, _ = env.step(action)
    env.render()
    state = next_state
    limit += 1

  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
